## C@R - Data Science Perspective

## Overview

This notebook works through the process of extracting data from an Excel spreadsheet, converting the data using the Resource Description Framework (RDF), loading RDF data into a database, semantically enriching the data using ontology languages, and finally demonstrating some flexible querying mechanisms.

### Technologies used in this notebook

Category | Technology | Link
-------- | ---------- | ----
User Interface | Jupyter | [Jupyter](http://jupyter.org) <br />
Raw Data | Excel Spreadsheet | [Excel Description](https://en.wikipedia.org/wiki/Microsoft_Excel) <br />
Database | Virtuoso Open-Source | [Virtuoso GitHub](https://github.com/openlink/virtuoso-opensource) <br />
Resource Description | RDF | [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework) <br />
Ontology Description | RDFS / OWL | [RDFS](https://en.wikipedia.org/wiki/RDF_Schema) / [OWL](https://en.wikipedia.org/wiki/Web_Ontology_Language) <br />
RDF Conversion Utility | csv2rdf | [RDFLib GitHub](https://github.com/RDFLib/rdflib/blob/master/rdflib/tools/csv2rdf.py) <br />
Query Language | SPARQL | [SPARQL](https://en.wikipedia.org/wiki/SPARQL) <br />
Programming Language | Python3 | [Python3](https://www.python.org) <br />
Data Handling | Pandas | [Pandas](http://pandas.pydata.org) <br />
SPARQL Wrapper | SPARQLWrapper | [SPARQLWrapper GitHub](https://github.com/RDFLib/sparqlwrapper) <br />



### Conversion and uploading of data into the database / triple store

Get the filename of the Excel spreadsheet:

In [1]:
!ls data/communities/*.xlsm

data/communities/DNL_Property_Impact_Estimator_V3.8.xlsm


Use pandas to load the spreadsheet and list worksheet names:

In [2]:
import pandas as pd

In [3]:
crdata = pd.ExcelFile("data/communities/DNL_Property_Impact_Estimator_V3.8.xlsm")

In [4]:
crdata.sheet_names

['User Guide',
 'Gauge-Console',
 'LRF',
 'GaugeLevels',
 'TA_Summary',
 'PP_Existing',
 'TA_Infrastructure',
 'Gauge_Infrastructure',
 'Agriculture',
 'Version Record']

Two interesting worksheets are 'GaugeLevels' and 'PP_Existing' that give a lot of data about gauging stations and properties respectively. We'll use these two worksheets in the rest of the notebook.

Load the gauge worksheet into a pandas dataframe and get overview of column names:

In [5]:
gauge_df = crdata.parse('GaugeLevels', header=1)

In [6]:
gauge_df.columns

Index(['Site_Reference', 'Station_Name', 'Display', 'Area', 'Model_Detail',
       'Station_Model', 'Station_Node', 'Model_Type', 'Gauge_Type',
       'Gauge_Datum', 'Q2', 'Q5', 'Q10', 'Q20', 'Q50', 'Q75', 'Q100', 'Q200',
       'Q1000', 'ModelledMin', 'ModelledMax', 'GaugeMin', 'MapInterval'],
      dtype='object')

In [7]:
gauge_df.head()

,Site_Reference,Station_Name,Display,Area,Model_Detail,Station_Model,Station_Node,Model_Type,Gauge_Type,Gauge_Datum,...,Q20,Q50,Q75,Q100,Q200,Q1000,ModelledMin,ModelledMax,GaugeMin,MapInterval
0,4023,ASHFORD,True,DNL,2D,River Wye SFRM Study_ Halcrow_ August 2010,AA5D,Level,Level,140.05,...,1.32,1.44,1.49,1.53,1.63,1.98,1.30,1.98,1.0,0.05
1,4091,BLYTH (RYTON),False,DNL,1D,River Ryton Flood Risk Mapping Study_ JBA_ Mar...,RYTO01_09697,Depth,Level,8.00,...,2.05,2.20,2.25,2.31,NaN,2.75,1.82,2.75,1.5,0.10
2,4203,BULWELL,False,DNL,1D/2D,River Leen (QMC) and Day Brook Study_ Black an...,130,Depth/Level,Level,46.00,...,1.52,1.80,1.90,2.09,2.39,2.84,1.26,2.84,1.0,0.10
3,4197,BUXTON,True,DNL,2D,River Wye SFRM Study_ Halcrow_ August 2010,WYA125ID_1i2,Level,Level,278.00,...,2.28,2.38,2.39,2.39,2.39,2.64,1.67,2.64,1.3,0.10
4,4043,CHATSWORTH,False,DNL,2D,River Derwent Recalibration_ Black and Veatch_...,DE179,Level,Level,99.00,...,4.52,4.87,5.18,5.28,5.57,NaN,3.86,5.57,3.2,0.10


Load the property worksheet into a pandas dataframe and get overview of column names:

In [8]:
property_df = crdata.parse('PP_Existing')

In [9]:
property_df.columns

Index(['JBAPropRef', 'Area', 'Gauge_PolyID', 'Gauge_SiteRef', 'Station_Name',
       'stationFW', 'GaugeDefence_Level', 'PropertyDefence_Level',
       'JBAPropertyType', 'JBAFloorLevel', 'geodb_oid', 'toid', 'bngeast',
       'bngnorth', 'organisation', 'bldgnumber', 'bldgname', 'subbldgname',
       'thoroughfare', 'posttown', 'postcode', 'os_class', 'MastermapToid',
       'topotoid', 'topotoidlz', 'topofid', 'floorarea', 'floorlevel',
       'mcmcode', 'housetype', 'FrismSourceID', 'Model', 'ModelSource', 'FZ2',
       'FZ3', 'ABD', 'LRF_NAME', 'fwd_tacode', 'fwa_name', 'Q_0gAUGE',
       'Q2_Gauge', 'Q5_Gauge', 'Q10_Gauge', 'Q20_Gauge', 'Q50_Gauge',
       'Q75_Gauge', 'Q100_Gauge', 'Q200_Gauge', 'Q1000_Gauge',
       'SurveyThreshold', 'LiDAR_2m_Min', 'LiDAR_2m_Mean', 'LiDAR_2m_Max',
       'Building_Thresh', 'Threshold_Source', 'Property_Thresh',
       'Q0_ExistingLevel', 'Q2_Existing Level Mean', 'Q5_Existing Level Mean',
       'Q10_Existing Level Mean', 'Q20_Existing Level M

In [10]:
property_df.head()

,JBAPropRef,Area,Gauge_PolyID,Gauge_SiteRef,Station_Name,stationFW,GaugeDefence_Level,PropertyDefence_Level,JBAPropertyType,JBAFloorLevel,...,Model Level known Max,Predicted_Level,Predicted_PropertyDepth,Defended_Depth,Protected,Damages Known Min,Damages Known Max,Predicted_Damage,Defended_Damage,Damage_Avoided
0,1,DNL,NaN,4203,BULWELL,BULWELL_,NaN,NaN,Residential,Ground Floor,...,NaN,0.0,0.0,0.0,False,NaN,NaN,0.0,0.0,0
1,2,DNL,NaN,4203,BULWELL,BULWELL_,NaN,NaN,Residential,Ground Floor,...,NaN,0.0,0.0,0.0,False,NaN,NaN,0.0,0.0,0
2,3,DNL,NaN,4203,BULWELL,BULWELL_,NaN,NaN,Residential,Upper Floor,...,NaN,0.0,0.0,0.0,False,NaN,NaN,0.0,0.0,0
3,4,DNL,NaN,4203,BULWELL,BULWELL_,NaN,NaN,Residential,Ground Floor,...,NaN,0.0,0.0,0.0,False,NaN,NaN,0.0,0.0,0
4,5,DNL,NaN,4203,BULWELL,BULWELL_,NaN,NaN,Residential,Ground Floor,...,NaN,0.0,0.0,0.0,False,NaN,NaN,0.0,0.0,0


Save the gauge and property worksheets to csv files for subsequent processing:

In [ ]:
gauge_df.to_csv('data/communities/gauge.csv')

In [11]:
!ls data/communities

DNL_Property_Impact_Estimator_V3.8.xlsm load_gauge_graph
csv2rdf.py                              load_property_graph
gauge.csv                               property.csv
gauge_ds.ttl                            property_ds.ttl


In [ ]:
property_df.to_csv('data/communities/property.csv')

In [12]:
!ls data/communities

DNL_Property_Impact_Estimator_V3.8.xlsm load_gauge_graph
csv2rdf.py                              load_property_graph
gauge.csv                               property.csv
gauge_ds.ttl                            property_ds.ttl


We can check for missing data - e.g. in the gauge worksheet

In [13]:
gauge_df = pd.read_csv('data/communities/gauge.csv')
gauge_df[['Station_Model'] + list(gauge_df.columns[gauge_df.isnull().any()])]

,Station_Model,Q2,Q5,Q10,Q50,Q75,Q200,Q1000
0,River Wye SFRM Study_ Halcrow_ August 2010,NaN,1.300,1.31,1.44,1.49,1.63,1.980
1,River Ryton Flood Risk Mapping Study_ JBA_ Mar...,NaN,1.820,1.94,2.20,2.25,NaN,2.750
2,River Leen (QMC) and Day Brook Study_ Black an...,NaN,1.260,1.37,1.80,1.90,2.39,2.840
3,River Wye SFRM Study_ Halcrow_ August 2010,NaN,1.670,1.95,2.38,2.39,2.39,2.640
4,River Derwent Recalibration_ Black and Veatch_...,NaN,NaN,3.86,4.87,5.18,5.57,NaN
5,River Meden Flood Risk Mapping Study_ JBA_ Jun...,NaN,1.010,1.06,1.18,1.21,1.25,1.410
6,River Derwent Confluence SFRM_ JBA_ July 2011,NaN,NaN,2.35,2.43,2.45,2.58,2.690
7,Greater Nottingham SFRA_ Existing Scenario_ Bl...,NaN,NaN,NaN,NaN,NaN,NaN,6.820
8,Flood Modelling of the River Smite_JBA_March 2012,NaN,2.170,2.29,2.44,2.46,2.55,2.710
9,Greater Nottingham SFRA_ Existing Scenario_ Bl...,NaN,4.830,NaN,NaN,NaN,NaN,6.230


And we can identify which studies don't use a Q200 return period

In [14]:
mask = gauge_df['Q200'].isnull()
gauge_df[mask][['Station_Model', 'Q2', 'Q5', 'Q10', 'Q50', 'Q75', 'Q200', 'Q1000']]

,Station_Model,Q2,Q5,Q10,Q50,Q75,Q200,Q1000
1,River Ryton Flood Risk Mapping Study_ JBA_ Mar...,NaN,1.82,1.94,2.20,2.25,NaN,2.75
7,Greater Nottingham SFRA_ Existing Scenario_ Bl...,NaN,NaN,NaN,NaN,NaN,NaN,6.82
9,Greater Nottingham SFRA_ Existing Scenario_ Bl...,NaN,4.83,NaN,NaN,NaN,NaN,6.23
17,River Sence Hydraulic Model and Floodplain Map...,NaN,3.14,3.26,3.48,NaN,NaN,NaN
30,River Idle Flood Risk Mapping Study_ JBA_ Marc...,NaN,1.04,1.17,1.46,NaN,NaN,NaN
32,River Erewash SFRM2 Study_ Hyder_ 2013,NaN,2.02,2.19,2.53,2.55,NaN,3.30
34,River Erewash SFRM2 Study_ Hyder_ 2013,NaN,1.52,1.72,2.28,2.35,NaN,3.66
36,River Soar Flood Risk Mapping Study_ Mott Macd...,NaN,1.70,1.90,2.20,NaN,NaN,NaN
50,River Ryton Flood Risk Mapping Study_ JBA_ Mar...,NaN,1.26,1.45,1.78,1.84,NaN,2.24


In [ ]:
return_periods = gauge_df[['Q2', 'Q5', 'Q10', 'Q50', 'Q75', 'Q200', 'Q1000']].isnull()
for return_period in return_periods:
   print(return_periods[return_period].value_counts())

In [ ]:
property_df = pd.read_csv('data/communities/property.csv')

In [ ]:
property_df.columns

In [ ]:
np = property_df.iloc(property_df['posttown'])

In [ ]:
np

We can now convert the csv files to triple format using the csv2rdf utility. RDF namespaces for subject base names and property basenames. The resulting RDF files are in [Turtle](https://www.w3.org/TR/turtle/) syntax.

In [ ]:
# http://ensembleprojects.org/ds/ns/floodrisk/gauge#
# http://ensembleprojects.org/ds/ns/floodrisk/gauge_data#

!python data/communities/csv2rdf.py -b http://ensembleprojects.org/ds/ns/floodrisk/gauge# -p http://ensembleprojects.org/ds/ns/floodrisk/gauge_data# -o data/communities/gauge_ds.ttl data/communities/gauge.csv

We can now take a quick look at the resulting triples and check the namespaces have been generated correctly: 

In [ ]:
!head data/communities/gauge_ds.ttl

Now generate the property triples:

In [ ]:
# http://ensembleprojects.org/ds/ns/floodrisk/property#
# http://ensembleprojects.org/ds/ns/floodrisk/property_data#

!python data/communities/csv2rdf.py -b http://ensembleprojects.org/ds/ns/floodrisk/property# -p http://ensembleprojects.org/ds/ns/floodrisk/property_data# -o data/communities/property_ds.ttl data/communities/property.csv

In [ ]:
!ls data/communities/*.ttl

We can now load the generated files into the Virtuoso database. Virtuoso uses trusted directories for uploading of data, so the ttl files care copied there:

In [ ]:
# Copy ttl files to allowed Virtuoso import directory
!cp data/communities/gauge_ds.ttl /usr/local/Cellar/virtuoso/7.2.4.2/share/virtuoso/vad/
!cp data/communities/property_ds.ttl /usr/local/Cellar/virtuoso/7.2.4.2/share/virtuoso/vad/


We can now load the files into Virtuoso using the isql interface and two small batch files. The triples are loaded into two separate named graphs: (i) <http://ensembleprojects.org/ds/floodrisk/gauge> and (ii) <http://ensembleprojects.org/ds/floodrisk/gauge>:

In [ ]:
!cat data/communities/load_gauge_graph

In [ ]:
!cat data/communities/load_property_graph

In [ ]:
# Load gauge ttl files into Virtuoso named graphs
!isql localhost dba dba data/communities/load_gauge_graph

In [ ]:
# Load property ttl files into Virtuoso named graphs
!isql localhost dba dba data/communities/load_property_graph

### Querying of Semantic Data

Now the raw data has been converted into triple form and uploaded into the Virtuoso triple store, we can query it using the SPARQL language. In this case the SPARQL query is embedded in Python using the SPARQLWrapper package. We have created two separate named graphs so we can query across one or both of them:

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Create the SPARQL query as a string. To illustrate querying, we can query across 
# both named graphs using the 'FROM' clause or, as in this case, simply comment out
# one of the named graphs.

sparql_query = """
SELECT ?subject ?predicate ?object
FROM <http://ensembleprojects.org/ds/floodrisk/gauge>
#FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  ?subject ?predicate ?object
}
LIMIT 5
"""

# Virtuoso SPARQL endpoint
sparql_endpoint = "http://localhost:8890/sparql"
sparql = SPARQLWrapper(sparql_endpoint)

# Return results in JSON format
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["subject"]["value"], result["predicate"]["value"], result["object"]["value"])

### Semantically enriching the data using an ontology 

The existing property dataset has different damage estimations on a per building basis for a number of different return periods. A simple exmaple of semantically enriching the data is to model these different return periods as a class hierachy; we can then query either specific return periods or all return periods.

The first step is to find the RDF properties that relate to the different damage estimations. To do this, we use a regular expression based filter:

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
SELECT DISTINCT ?property
FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  ?s ?property ?o .
  FILTER regex(?property,'existingdamage','i')
}
"""

sparql_endpoint = "http://localhost:8890/sparql"
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result['property']['value'])

These look like the correct RDF properites for damage estimations. We can model these as a property hierachy with a top-level 'Q' return period and subsequent sub-properties for individual return periods.

We do this using the RDFS ontology languge. We declare 'Q' as a rdf:Property type and then declare the specific return period properties as a rdfs:subPropertyOf of 'Q'. In Turtle syntax:

Q a rdf:Property . <br />
q2_existingDamageMean rdfs:subPropertyOf Q . <br />
q5_existingDamageMean rdfs:subPropertyOf Q . <br />
... <br />


The SPARQL code below generates the triples representing this ontology and inserts them into the <http://ensembleprojects.org/ds/floodrisk/property> named graph.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
PREFIX g:       <http://ensembleprojects.org/ns/floodrisk/gauge#>
PREFIX gd:      <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
PREFIX p:       <http://ensembleprojects.org/ns/floodrisk/property#>
PREFIX pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>
PREFIX powl:    <http://ensembleprojects.org/owl/propertymodel#>
PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#> 

INSERT {
  powl:Q a rdf:Property . 
  ?p rdfs:subPropertyOf powl:Q 
}
FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  ?s ?p ?o .
  FILTER regex(?p,'existingdamage','i')
}
"""

sparql_endpoint = "http://localhost:8890/sparql"
sparql = SPARQLWrapper(sparql_endpoint)
# As we're updating the triple store, we need to use the 'POST' method
sparql.setMethod('POST')
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result)

Now the ontology triples are in the database, we need to tell Virtuoso to generate new triples using its inferencing engine. This is done through the Virtuoso isql interface:

$ isql <br />
SQL> rdfs_rule_set('http://ensembleprojects.org/ds/floodrisk/property',  'http://ensembleprojects.org/ds/floodrisk/property'); <br />
SQL>exit; <br />

In Virtuoso, we use the 'DEFINE'statement to give a custom inferencing context. We can then query damage estimations for all return periods using the generic 'Q' return period:

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
DEFINE input:inference <http://ensembleprojects.org/ds/floodrisk/property>
PREFIX p:       <http://ensembleprojects.org/ds/ns/floodrisk/property#>
PREFIX powl:    <http://ensembleprojects.org/owl/propertymodel#>

SELECT *
FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  p:0 powl:Q ?value .
}
"""
sparql_endpoint = "http://localhost:8890/sparql"
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["value"]['value'])

Although we can get all the return period damage estimations using the above method, in general we want to know both the values and the associated return periods:

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
DEFINE input:inference <http://ensembleprojects.org/ds/floodrisk/property>
PREFIX p:       <http://ensembleprojects.org/ds/ns/floodrisk/property#>
PREFIX pd:      <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
PREFIX powl:    <http://ensembleprojects.org/owl/propertymodel#>
PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#> 

SELECT ?returnPeriod ?value
FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  p:0 ?returnPeriod ?value .
  ?returnPeriod rdfs:subPropertyOf powl:Q
}
"""
sparql_endpoint = "http://localhost:8890/sparql"
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

returnPeriods = []
values = []
for result in results["results"]["bindings"]:
    returnPeriod = int(result['returnPeriod']['value'].split('#')[1].split('_')[0].split('q')[1])
    returnPeriods.append(returnPeriod)
    values.append(result['value']['value'])

vals = zip(returnPeriods, values)
sorted_vals = sorted(vals)
returnPeriods = [val[0] for val in sorted_vals]
values = [val[1] for val in sorted_vals]

print('Period\t\tDamage Estimation')
for i in range(len(returnPeriods)):
    print(returnPeriods[i], '\t\t', values[i])